# Graph Coloring

In [64]:
def ReadFile(file_location):
    with open(file_location, 'r') as input_data_file:
        input_data = input_data_file.read()
    return input_data

def GetInputs(input_data):
    input_data = input_data.splitlines()
    input_data = [list(map(int, x.split(' ')))  for x in input_data]
    inputs={}
    inputs['nbnodes'] = input_data[0][0]
    inputs['nbedges'] = input_data[0][1]
    #inputs['startnodes'] = [row[0] for row in input_data[1:]]
    #inputs['endnodes'] = [row[1] for row in input_data[1:]]
    adjacentnodes = {}
    for row in input_data[1:]:
        node1 = row[0]
        node2 = row[1]
        if node1 not in adjacentnodes:
            adjacentnodes[node1] = []
        if node2 not in adjacentnodes[node1]:
            adjacentnodes[node1].append(node2)
        if node2 not in adjacentnodes:
            adjacentnodes[node2] = []
        if node1 not in adjacentnodes[node2]:
            adjacentnodes[node2].append(node1)
    for k, v in adjacentnodes.items():
        v.sort()
    inputs['nodes'] = list(adjacentnodes.keys())
    inputs['colors'] = range(len(inputs['nodes'])) # initialize colors to one for each node
    inputs['adjacentnodes'] = adjacentnodes
    return inputs

inputs = GetInputs(ReadFile('data/gc_1000_5'))
#inputs

### Option 1: Greedy algorithm

In [68]:
%%time

nodecolors = {}

# loop on each node
for node in inputs['nodes']:
    #print('>> Node ', node)
    for color in inputs['colors']:
        #print('    >> Color ', color)
        # try to assign each color until one is valid
        # to determine if a color is valid, we look at all adjacent nodes and discard already assigned colors.
        isAvailable = True
        
        if node not in inputs['adjacentnodes']:
            # if the node doesn't have any adjacent nodes, assign the first available color
            print('does not have any neighboors')
            nodecolors[node] = color
            break
        
        adjacentNodes = inputs['adjacentnodes'][node]
        for adjacentNode in adjacentNodes:
            if adjacentNode in nodecolors:
                # if the node already has a color assigned ...
                if nodecolors[adjacentNode] == color:
                    # ... and if the color assigned is the same than the current one, discard it.
                    isAvailable = False
                    break
        
        if isAvailable:
            nodecolors[node] = color
            break

output = {}
output['objective'] = len(set(nodecolors.values()))
output['variables'] = list(nodecolors.values())
output

Wall time: 741 ms
